### **01 SILVER PRELOAD CONFIG - SAP**

In [2]:
from pyspark.sql.functions import col
from datetime import datetime
import json

# Set the configuration
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY")
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

StatementMeta(, fe4c9894-151c-4614-b769-243f352a6001, 4, Finished, Available, Finished)

In [ ]:
# Parameterised Bronze Path 
try:
    p_workSpace = workSpace
    p_accountName = accountName
    p_relativePath = relativePath
except:
    p_workSpace = "a3e4d0e6-6b32-498b-addc-1629a4b29a47"
    p_accountName = "onelake"
    p_relativePath = "62139ac1-f383-41dc-b7a5-64655894ccd2/Files/bronze"

print(f"p_workSpace: {p_workSpace}")
print(f"p_accountName: {p_accountName}")
print(f"p_relativePath: {p_relativePath}")

try:
    p_db_bronzeFolder = json.loads(p_db_bronzeFolder)
    dbname = f'["06_{p_db_bronzeFolder}"]'

except:
    dbname = ["XXX_SAP"]

try:
    p_todaysFile = p_todaysFile
except:
    # today = datetime.today()
# Format the string
    # p_todaysFile = today.strftime("Year_%Y/Month_%m/Day_%d")
    p_todaysFile = "Year_2025/Month_06/Day_02"
    
print(f"p_db_bronzeFolder: {dbname}")
print(f"p_todaysFile: {p_todaysFile}")


In [ ]:
try:
    db_bronzeFolder = json.loads(db_bronzeFolder)
    # db_bronzeFolder = ["STORM", "SAP", "BAIL"]
except:
    db_bronzeFolder = ["06_SAP"]
# File to be processed today
try:
    todaysFile = todaysFile
except:
# Get today's date
    # today = datetime.today()
# Format the string
    # todaysFile = today.strftime("Year_%Y/Month_%m/Day_%d")
    todaysFile = "Year_2025/Month_06/Day_05"
    
print(db_bronzeFolder)
print(todaysFile)


try:
    p_db_bronzeFolder = [f"06_{p_db_bronzeFolder}"]
except:
    p_db_bronzeFolder = ["06_SAP"]

try:
    p_todaysFile = todaysFile
except:
    # today = datetime.today()
# Format the string
    # p_todaysFile = today.strftime("Year_%Y/Month_%m/Day_%d")
    p_todaysFile = "Year_2025/Month_06/Day_06"
    
print(f"p_db_bronzeFolder: {p_db_bronzeFolder}")
print(f"p_todaysFile: {p_todaysFile}")


In [4]:
folder_path = f"abfss://{p_workSpace}@{p_accountName}.dfs.fabric.microsoft.com/{p_relativePath}/{p_db_bronzeFolder[0]}/{p_todaysFile}"

print(folder_path)

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, 6, Finished, Available, Finished)

NameError: name 'workSpace' is not defined

In [ ]:

db_bronzePath = "abfss://%s@%s.dfs.fabric.microsoft.com/%s" % (p_workSpace, p_accountName, p_relativePath)
print(db_bronzePath)

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [ ]:
# Create Bronze file path
# db_bronzeFile_Path = [f"{db_bronzePath}/{path}/{p_todaysFile}" for path in p_db_bronzeFolder]
# print(bronzeFile_Path)
# # spark

db_bronzeFile_Path = [f"{db_bronzePath}/{path}/{p_todaysFile}" for path in dbname]
print(db_bronzeFile_Path)
# abfss://a3e4d0e6-6b32-498b-addc-1629a4b29a47@onelake.dfs.fabric.microsoft.com/62139ac1-f383-41dc-b7a5-64655894ccd2/Files/bronze/06_SAP

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [ ]:
try:
    # df = spark.read.parquet("abfss://a3e4d0e6-6b32-498b-addc-1629a4b29a47@onelake.dfs.fabric.microsoft.com/62139ac1-f383-41dc-b7a5-64655894ccd2/Files/bronze/04_STORM/Year_2025/Month_05/Day_31/LOCATION_2025-05-23 11_59_35.parquet")
    df2 = spark.read.parquet("abfss://a3e4d0e6-6b32-498b-addc-1629a4b29a47@onelake.dfs.fabric.microsoft.com/62139ac1-f383-41dc-b7a5-64655894ccd2/Files/bronze/04_STORM/Year_2025/Month_05/Day_31/FORCE_STRUCTURE_LOC_2025-03-27 10_14_52.parquet")
    # df.printSchema()
    df2.printSchema()
except Exception as e:
    print("Schema read failed:", str(e))


StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

#### **FUNCTIONS**

##### Temp Files

In [ ]:
# Process each path
for path in db_bronzeFile_Path:
    try:
        # Get parquet files
        parquetFiles = [files.path for files in mssparkutils.fs.ls(path) if files.name.endswith(".parquet")]
        
        if not parquetFiles:
            print(f"No parquet files found in {path}")
            continue
            
        # Process each parquet file
        for filePath in parquetFiles:
            try:
                # Extract table name from file path
                fileName = filePath.split("/")[-1].replace(".parquet", "")
                tableName = fileName.rsplit("_")[0]  # remove dates
                
                # Read and create temp view
                df = spark.read.parquet(filePath)
                df.createOrReplaceTempView(f"temp_{tableName}")
                
                print(f"\nSuccessfully created temp view: temp_{tableName} -=- {df.count()} Records")
                # print(f"From file: {filePath}")
                
            except Exception as e:
                print(f"Error processing file {filePath}: {str(e)}")
                
    except Exception as e:
        print(f"Error accessing path {path}: {str(e)}")


StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

##### Tables

In [ ]:
def createTable(createtable):
    for tableName, sql in createtable:
        if spark.catalog.tableExists(tableName):
            print(f" 🕑 Skilpping .... Table {tableName} Exit Already")
        else:
            try:
                print(f"🕑 Creating Table... -=- {tableName} -=-")
                spark.sql(sql)
                print(f"✅ {tableName} -=- Created Successfully\n")
            except Exception as e:
                print(f"❌ Table {tableName} Creation FAILED !\n {str(e)}")

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

### **BRONZE TO SILVER TABLES**

###### Creating Tables

In [ ]:
# CHANGE SAP
presonnel = createTable([
# -- personnel
    ("tbl_storm_personnel",
    """CREATE TABLE IF NOT EXISTS tbl_storm_personnel(
        PERSONNELID INT
        ,AGENCYCODE STRING
        ,EMPLOYEENO STRING
        ,BADGENO INT
        ,FAMILYNAME STRING
        ,FIRSTNAME STRING
        ,P_RANK STRING
        ,DEPARTMENT STRING
        ,TEAMSCHEDULE STRING
        ,SECURITYGROUP STRING
        ,ACTIVE INT
        ,MODDATE STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """),

# -- location   
    ("tbl_storm_location",
        """CREATE TABLE IF NOT EXISTS tbl_storm_location(
        LOCATIONID INT
        ,LOCATIONSEGMENT STRING
        ,DISTRICTNAME STRING
        ,CITY STRING
        ,LOCATIONNAME STRING
        ,LOCATIONENVIRONMENT STRING
        ,POSTCODE STRING
        ,LONGITUDE INT
        ,LATITUDE INT
        ,DATEMOD STRING
        ,TIMEMOD STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        ) USING DELTA;
    """),
# -- servicecode
    ("tbl_storm_service_code",
        """CREATE TABLE IF NOT EXISTS tbl_storm_service_code(
        SERVICECODENATURALID STRING
        ,SERVICECODE STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """),
    
# -- priority
    ("tbl_storm_priority",
    """CREATE TABLE IF NOT EXISTS tbl_storm_priority(
        PRIORITYID INT
        ,PRIORITY INT
        ,TARGETRESPONSETIME INT
        ,TARGETALLOCATIONTIME INT
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """),

# -- force_strictire_loc
    ("tbl_storm_force_structure_loc",
    """CREATE TABLE IF NOT EXISTS tbl_storm_force_structure_loc(
        GEONATURALID STRING
        ,LPACODE STRING
        ,LPADESCRIPTION STRING
        ,DPACODE STRING
        ,DPADESCRIPTION STRING
        ,DIVISIONCODE STRING
        ,DIVISIONDESCRIPTION STRING
        ,DISTRICTCODE STRING
        ,DISTRICTDESCRIPTION STRING
        ,NEIGHBOURHOODCODE STRING
        ,NEIGHBOURHOODDESCRIPTION STRING
        ,NEIGHBOURHOODID STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """),

# -- unit_activity
    ("tbl_storm_unit_activity",
    """CREATE TABLE IF NOT EXISTS tbl_storm_unit_activity(
        ISR_NUMBER STRING
        ,UNIT STRING
        ,P_UNIT_TYPE1 STRING
        ,PLAN_NUMBER STRING
        ,UA_DATE DATE
        ,UA_TIME STRING
        ,SEQ_NO STRING
        ,OFFICER_PERS_ID1 STRING
        ,OFFICER_PERS_ID2 STRING
        ,OFFICER_PERS_ID3 STRING
        ,OFFICER_PERS_ID4 STRING
        ,OFFICER_PERS_ID5 STRING
        ,OFFICER_PERS_ID6 STRING
        ,OFFICER_PERS_ID7 STRING
        ,OFFICER_PERS_ID8 STRING
        ,OFFICER_PERS_ID9 STRING
        ,OFFICER_PERS_ID10 STRING
        ,OFFICER_PERS_ID11 STRING
        ,OFFICER_PERS_ID12 STRING
        ,RIDERS STRING
        ,PRIORITY STRING
        ,S_STATUS STRING
        ,S_FUNCTION STRING
        ,DISPATCH_LEVEL STRING
        ,LONGITUDE INT
        ,LATITUDE INT
        ,MOD_DATE DATE
        ,MOD_TIME STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        ) USING DELTA;
    """),

# -- isr_timestamps
    ("tbl_storm_isr_timestamps",
    """CREATE TABLE IF NOT EXISTS tbl_storm_isr_timestamps(
        ISR_NO STRING
        ,MOD_DATE DATE
        ,MOD_TIME STRING
        ,TIMESTAMP7 TIMESTAMP
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """),

#  -- isr
    ("tbl_storm_isr",
    """CREATE TABLE IF NOT EXISTS tbl_storm_isr(
        ISR_NO	STRING
        ,DISPATCH_LEVEL	STRING
        ,LOC_ID	INT
        ,LOC_NAME	STRING
        ,LOC_TYPE	STRING
        ,LEVEL1	STRING
        ,LEVEL2	STRING
        ,LEVEL3	STRING
        ,LEVEL4	STRING
        ,LEVEL5	STRING
        ,FINAL_SERV_CODE	STRING
        ,INITIAL_SERV_CODE	STRING
        ,P_ORIGIN	STRING
        ,PRIORITY	INT
        ,HANDLING_UNIT	STRING
        ,DP_ID	INT
        ,DP_BADGE_NO	STRING
        ,SUPER_ID	INT
        ,SUPER_BADGE_NO	STRING
        ,OFF_ID1	INT
        ,OFF_ID2	INT
        ,OFF_BADGE_NO1	STRING
        ,OFF_BADGE_NO2	STRING
        ,S_STATUS	STRING
        ,S_STATUS_NOTE	STRING
        ,LONGITUDE	INT
        ,LATITUDE	INT
        ,PERSON	INT
        ,LOG	INT
        ,MOD_LOC	INT
        ,MOD_SERV_CODE	INT
        ,MOD_PRIORITY	INT
        ,CREA_LOGINNAME	INT
        ,CREA_PERSONNEL_ID	INT
        ,MOD_DATE	DATE
        ,MOD_TIME	STRING
        ,OIC_BADGE_NO	STRING
        ,OIC_ID	STRING
        ,ORIGINAL_PRIORITY	INT
        ,STATUS1_DATE	DATE
        ,STATUS1_TIME	STRING
        ,STATUS2_DATE	DATE
        ,STATUS2_TIME	STRING
        ,STATUS3_DATE	DATE
        ,STATUS3_TIME	STRING
        ,STATUS4_DATE	DATE
        ,STATUS4_TIME	STRING
        ,STATUS5_DATE	DATE
        ,STATUS5_TIME	STRING
        ,STATUS6_DATE	DATE
        ,STATUS6_TIME	STRING
        ,source_table	STRING
        ,CRITICALITY	STRING
        ,DISPOSE_CODE1	STRING
        ,VSS	STRING
        ,OA_REC_PROCESSED_DATE TIMESTAMP
        ,OA_RECORDID STRING
        )USING DELTA;
    """)
])


StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [ ]:



sap = createTable([
# -- -- SAP_HRP1000
    (
"SAP_HRP1000",
    """CREATE TABLE IF NOT EXISTS SAP_HRP1000(
    OBJ_ID STRING
    ,OBJ_TYPE STRING
    ,OBJ_DESC STRING
    ,START_DATE DATE
    ,END_DATE DATE
    ,OA_REC_PROCESSED_DATE TIMESTAMP
    ,OA_RECORDID STRING
    )USING DELTA;
"""),

# -- SAP_HRP1001
    (
"SAP_HRP1001",
    """CREATE TABLE IF NOT EXISTS SAP_HRP1001(
    OBJ_ID STRING
    ,OBJ_TYPE STRING
    ,OBJ_DESC STRING
    ,RELAT STRING
    ,TYPE_OF_RELATED_OBJ STRING
    ,ID_OF_RELATED_OBJ STRING
    ,RELAT_SPEC STRING
    ,START_DATE DATE
    ,END_DATE DATE
    ,OA_REC_PROCESSED_DATE TIMESTAMP
    ,OA_RECORDID STRING
    )USING DELTA;
"""),

# -- SAP_PA0001
(
"SAP_PA0001",
    """CREATE TABLE IF NOT EXISTS SAP_PA0001(
    PERSONNEL_NUMBER INT
    ,EE_SUBGROUP VARCHAR(2)
    ,ORG_UNIT INT
    ,POSITION INT
    ,JOB INT
    ,NAME_SOFTWARE VARCHAR(40)
    ,NAME_FORMATTED VARCHAR(40)
    ,OBJ_TYPE VARCHAR(2)
    ,START_DATE DATE
    ,END_DATE DATE
    ,OA_REC_PROCESSED_DATE TIMESTAMP
    ,OA_RECORDID STRING
    )USING DELTA;
"""),

# -- TBL_SAP_ORG
(
"TBL_SAP_ORG",
    """CREATE TABLE IF NOT EXISTS TBL_SAP_ORG(
    PERSONNELNUMBER	STRING
    ,STARTDATE	DATE
    ,ENDDATE	DATE
    ,FULLNAME	STRING
    ,SECTION	STRING
    ,SECTIONID	STRING
    ,UNIT	STRING
    ,UNITID	STRING
    ,LOCATION	STRING
    ,LOCATIONID	STRING
    ,DEPARTMENT	STRING
    ,DEPARTMENTID	STRING
    ,DIRECTORATE	STRING
    ,DIRECTORATEID	STRING
    ,COMMAND	STRING
    ,COMMANDID	STRING
    ,FORCE	STRING
    ,FORCEID	STRING
    ,CURRENT	INT
    ,OA_REC_PROCESSED_DATE TIMESTAMP
    ,OA_RECORDID STRING
    )USING DELTA;
"""),

# -- sap_staff_history
(
"TBL_SAP_STAFF_HISTORY",
    """CREATE TABLE IF NOT EXISTS TBL_SAP_STAFF_HISTORY(
    PERSONNELNUMBER INT
    ,STARTDATE DATE
    ,ENDDATE DATE
    ,FULLNAME VARCHAR(50)
    ,ROLE VARCHAR(50)
    ,ROLEID VARCHAR(50)
    ,SECTION VARCHAR(50)
    ,SECTIONID VARCHAR(50)
    ,UNIT VARCHAR(50)
    ,UNITID VARCHAR(50)
    ,LOCATION VARCHAR(50)
    ,LOCATIONID VARCHAR(50)
    ,DEPARTMENT VARCHAR(50)
    ,DEPARTMENTID VARCHAR(50)
    ,COMMAND VARCHAR(50)
    ,COMMANDID VARCHAR(50)
    ,DIRECTORATE VARCHAR(50)
    ,DIRECTORATEID VARCHAR(50)
    ,FORCE VARCHAR(50)
    ,FORCEID VARCHAR(50)
    ,STAFFMOVEID VARCHAR(50)
    ,CURRENTFLAG VARCHAR(50)
    ,OA_REC_PROCESSED_DATE TIMESTAMP
    ,OA_RECORDID STRING
    )USING DELTA;
""")

])

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [14]:
 %%sql
WITH pa0001_cast AS (
    SELECT 
        TRY_CAST(pernr AS STRING) AS PersonnelNumber,
        TRY_CAST(begda AS INT) AS start_date_int,
        TRY_CAST(endda AS INT) AS end_date_int,
        TRY_CAST(orgeh AS STRING) AS orgeh,
        TRY_CAST(ename AS STRING) AS FullName,
        TRY_CAST(plans AS STRING) AS plans,
        ROW_NUMBER() OVER(PARTITION BY pernr ORDER BY endda DESC)  AS ROWCOUNT  --pick the most recent record
    FROM temp_PA0001
)  
,pa0001_cast_current AS (
    SELECT *
    FROM pa0001_cast WHERE ROWCOUNT =1
-- limit 15;
)

-- SELECT * FROM pa0001_cast_current --WHERE ROWCOUNT =1
-- limit 15;


,pa0001_filtered AS (
    SELECT 
        PersonnelNumber,
        start_date_int,
        end_date_int,
        orgeh,
        FullName,
        plans
    FROM (
        SELECT 
            *,
            ROW_NUMBER() OVER (PARTITION BY PersonnelNumber ORDER BY end_date_int DESC) AS rn
        FROM pa0001_cast_current 
        WHERE start_date_int <= YEAR(CURRENT_DATE) * 10000 + MONTH(CURRENT_DATE) * 100 + DAY(CURRENT_DATE)
    )
    WHERE start_date_int >= 20000101
    AND rn = 1
)

-- SELECT * FROM pa0001_filtered limit 15;
,hrp1000_data AS (
    SELECT 
        TRY_CAST(objid AS STRING) AS objid,
        TRY_CAST(stext AS STRING) AS stext 
    FROM temp_hrp1000
    WHERE (otype IN ('S', 'BA', 'C', 'O', 'L'))
    AND endda = 99991231
)
-- SELECT * FROM hrp1000_data LIMIT 15;

,hrp1001_data AS (
    SELECT 
        TRY_CAST(begda AS INT) AS begda,
        TRY_CAST(endda AS INT) AS endda,
        TRY_CAST(objid AS STRING) AS objid,
        TRY_CAST(sobid AS STRING) AS sobid
        -- ROW_NUMBER() OVER(PARTITION BY sobid ORDER BY endda DESC)  AS ROWCOUNT
    FROM temp_hrp1001
    WHERE endda = 99991231
)
-- SELECT *
-- FROM hrp1001_data
-- LIMIT 15;
,org_structure AS (
    SELECT 
        p.PersonnelNumber,
        p.start_date_int,
        p.end_date_int,
        p.FullName,
        org.stext AS section,
        TRY_CAST(p.orgeh AS INT) AS sectionid,
        unit.stext AS unit,
        TRY_CAST(unit.objid AS INT) AS unitid,
        loc.stext AS location,
        TRY_CAST(loc.objid AS INT) AS locationid,
        dept.stext AS department,
        TRY_CAST(dept.objid AS INT) AS departmentid,
        cmd.stext AS command,
        TRY_CAST(cmd.objid AS INT) AS commandid,
        dir.stext AS directorate,
        TRY_CAST(dir.objid AS INT) AS directorateid,
        force.stext AS force,
        TRY_CAST(force.objid AS INT) AS forceid,
        CASE
            WHEN (YEAR(CURRENT_DATE) * 10000 + MONTH(CURRENT_DATE) * 100 + DAY(CURRENT_DATE)) 
                 BETWEEN p.start_date_int AND p.end_date_int
                 AND p.plans NOT IN ('00000000', '99999999') THEN 1 
            ELSE 0 
        END AS current
    FROM pa0001_filtered p
    JOIN hrp1000_data org ON p.orgeh = org.objid
    JOIN hrp1001_data rel1 ON org.objid = rel1.objid
    JOIN hrp1000_data unit ON rel1.sobid = unit.objid
    JOIN hrp1001_data rel2 ON unit.objid = rel2.objid
    JOIN hrp1000_data loc ON rel2.sobid = loc.objid
    JOIN hrp1001_data rel3 ON loc.objid = rel3.objid
    JOIN hrp1000_data dept ON rel3.sobid = dept.objid
    JOIN hrp1001_data rel4 ON dept.objid = rel4.objid
    JOIN hrp1000_data cmd ON rel4.sobid = cmd.objid
    JOIN hrp1001_data rel5 ON cmd.objid = rel5.objid
    JOIN hrp1000_data dir ON rel5.sobid = dir.objid
    JOIN hrp1001_data rel6 ON dir.objid = rel6.objid
    JOIN hrp1000_data interim ON rel6.sobid = interim.objid
    JOIN hrp1001_data rel7 ON interim.objid = rel7.objid
    JOIN hrp1000_data force ON rel7.sobid = force.objid
    WHERE p.end_date_int >= 20150101 -- check with Lewis if this might change


)
select * from org_structure limit 1000;

/*
,org_structure_select AS(SELECT --DISTINCT
    PersonnelNumber,
    TO_DATE(
        CONCAT(
            SUBSTRING(CAST(start_date_int AS STRING), 1, 4), '-',
            SUBSTRING(CAST(start_date_int AS STRING), 5, 2), '-',
            SUBSTRING(CAST(start_date_int AS STRING), 7, 2)
        ), 'yyyy-MM-dd'
    ) AS StartDate,
    TO_DATE(
        CONCAT(
            SUBSTRING(CAST(end_date_int AS STRING), 1, 4), '-',
            SUBSTRING(CAST(end_date_int AS STRING), 5, 2), '-',
            SUBSTRING(CAST(end_date_int AS STRING), 7, 2)
        ), 'yyyy-MM-dd'
    ) AS EndDate,
    FullName,
    section,
    sectionid,
    unit,
    unitid,
    location,
    locationid,
    department,
    departmentid,
    directorate,
    directorateid,
    command,
    commandid,
    force,
    forceid,
    current,
    ROW_NUMBER() OVER(PARTITION BY PersonnelNumber ORDER BY end_date_int DESC) AS hrp1001_duplicate,
    SHA2(CONCAT_WS('|',COALESCE(PersonnelNumber, ''), COALESCE(StartDate, ''),COALESCE(EndDate, ''), COALESCE(fullname, ''), COALESCE(sectionid, ''), COALESCE(unitid, ''), COALESCE(locationid, '')
    ,COALESCE(departmentid, '') , COALESCE(directorateid, ''), COALESCE(commandid, ''), COALESCE(forceid, '')),256) AS OA_RECORDID
FROM org_structure
    )
-- Final insert with proper date conversion
INSERT INTO tbl_sap_org
SELECT PersonnelNumber, StartDate, EndDate, FullName, section, sectionid, unit, unitid, location, locationid, department, departmentid, directorate, directorateid, command, commandid, force, forceid, current,
COALESCE(NULL, CURRENT_TIMESTAMP()), OA_RECORDID 
FROM org_structure_select 
WHERE hrp1001_duplicate = 1
*/




StatementMeta(, 1f077341-81e3-4699-87e0-6ae90d112a43, 16, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 19 fields>

In [ ]:

""" pick just this one officer (Mr Philip Smith). form this query: -- select * from org_structure limit 1000;
he has 15 Entries.
Do we need History or just the most recent record?
Their enddate is in the future but is_current is 0
EndDate defult to future
"""
# lewis for interpretation of data

# 10m records take between 16 -20 min to load daily. and if we don't need all why don't parameterised in SAP to get what we need first

# filter plvar = plan version in BOJ


StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [ ]:
%%sql
-- # # Ingest HRP1000
INSERT INTO sap_hrp1000(OBJ_ID,OBJ_TYPE,OBJ_DESC,START_DATE, END_DATE, OA_REC_PROCESSED_DATE, OA_RECORDID) 
              WITH hrp1000 AS(SELECT DISTINCT 
                TRY_CAST(OBJID AS VARCHAR(8)) AS OBJ_ID, 
                TRY_CAST(OTYPE AS VARCHAR(2)) AS OBJ_TYPE, 
                TRY_CAST(STEXT AS VARCHAR(50)) AS OBJ_DESC, 
                TO_DATE(SUBSTRING(BEGDA, 1, 4) || '-' || SUBSTRING(BEGDA, 5, 2) || '-' || SUBSTRING(BEGDA, 7, 2), 'yyyy-MM-dd') AS START_DATE, 
                TO_DATE(SUBSTRING(ENDDA, 1, 4) || '-' || SUBSTRING(ENDDA, 5, 2) || '-' || SUBSTRING(ENDDA, 7, 2), 'yyyy-MM-dd') AS END_DATE, 
                ROW_NUMBER() OVER(PARTITION BY OBJID ORDER BY ENDDA DESC) hrp1000_duplicate 
                ,LANGU
                ,SHA2(CONCAT_WS('|',COALESCE(OBJID, ''), COALESCE(OTYPE, ''),COALESCE(STEXT, '')),256) AS OA_RECORDID
                FROM temp_HRP1000 WHERE OBJID IS NOT NULL) 
            SELECT OBJ_ID, OBJ_TYPE,OBJ_DESC,START_DATE, END_DATE,COALESCE(NULL, CURRENT_TIMESTAMP()), OA_RECORDID FROM hrp1000 WHERE 1=1 AND (hrp1000_duplicate = 1 AND LANGU IN ('E'));


  -- # # Ingest HRP1001
 INSERT INTO sap_hrp1001(OBJ_ID, OBJ_TYPE, RELAT, TYPE_OF_RELATED_OBJ, ID_OF_RELATED_OBJ, RELAT_SPEC,  START_DATE, END_DATE, OA_REC_PROCESSED_DATE, OA_RECORDID) 
            WITH hrp1001 AS(SELECT DISTINCT 
            	 TRY_CAST(OBJID AS VARCHAR(8)) AS OBJ_ID 
            	,TRY_CAST(OTYPE AS VARCHAR(2)) AS OBJ_TYPE 
            	,TRY_CAST(RELAT AS VARCHAR(3)) AS RELAT  
            	,TRY_CAST(SCLAS AS VARCHAR(2)) AS TYPE_OF_RELATED_OBJ 
            	,TRY_CAST(SOBID AS VARCHAR(45)) AS ID_OF_RELATED_OBJ 
            	,TRY_CAST(RSIGN AS CHAR(1)) AS RELAT_SPEC 
              ,TO_DATE(SUBSTRING(BEGDA, 1, 4) || '_' + SUBSTRING(BEGDA, 5, 2) || '_' + SUBSTRING(BEGDA, 7, 2), 'yyyy-MM-dd') AS START_DATE 
            	,TO_DATE(SUBSTRING(ENDDA, 1, 4) || '_' + SUBSTRING(ENDDA, 5, 2) || '_' + SUBSTRING(ENDDA, 7, 2), 'yyyy-MM-dd') AS END_DATE 
              ,ROW_NUMBER() OVER(PARTITION BY OBJID ORDER BY OBJID) hrp1001_duplicate
              ,SHA2(CONCAT_WS('|',COALESCE(OBJID, ''), COALESCE(OTYPE, ''),COALESCE(RELAT, ''),COALESCE(SCLAS, ''),COALESCE(SOBID, ''),COALESCE(RSIGN, '')),256) AS OA_RECORDID
        FROM temp_hrp1001 WHERE OBJID IS NOT NULL) 
        SELECT OBJ_ID, OBJ_TYPE, RELAT, TYPE_OF_RELATED_OBJ, ID_OF_RELATED_OBJ, RELAT_SPEC,  START_DATE, END_DATE,COALESCE(NULL, CURRENT_TIMESTAMP()), OA_RECORDID FROM hrp1001 WHERE 1=1 AND hrp1001_duplicate = 1;

-- # # Ingest PA0001
INSERT INTO sap_pa0001 (PERSONNEL_NUMBER, EE_SUBGROUP, ORG_UNIT, POSITION, JOB, NAME_SOFTWARE, NAME_FORMATTED, OBJ_TYPE, START_DATE, END_DATE, OA_REC_PROCESSED_DATE, OA_RECORDID) 
            WITH pa0001 AS(SELECT DISTINCT 
        	    TRY_CAST(PERNR AS INT) AS PERSONNEL_NUMBER 
        	    ,TRY_CAST(PERSK AS VARCHAR(2)) AS EE_SUBGROUP 
        	    ,TRY_CAST(ORGEH AS INT) AS ORG_UNIT 
        	    ,TRY_CAST(PLANS AS INT) AS POSITION 
        	    ,TRY_CAST(STELL AS INT) AS JOB 
        	    ,TRY_CAST(SNAME AS VARCHAR(40)) AS NAME_SOFTWARE 
        	    ,TRY_CAST(ENAME AS VARCHAR(40)) AS NAME_FORMATTED 
        	    ,TRY_CAST(OTYPE AS VARCHAR(2)) AS OBJ_TYPE 
              ,TO_DATE(SUBSTRING(BEGDA, 1, 4) || '-' || SUBSTRING(BEGDA, 5, 2) || '-' || SUBSTRING(BEGDA, 7, 2), 'yyyy-MM-dd') AS START_DATE 
              ,TO_DATE(SUBSTRING(ENDDA, 1, 4) || '-' || SUBSTRING(ENDDA, 5, 2) || '-' || SUBSTRING(ENDDA, 7, 2), 'yyyy-MM-dd') AS END_DATE 
              ,ROW_NUMBER() OVER(PARTITION BY PERNR ORDER BY ENDDA DESC) pa0001_duplicate
              ,SHA2(CONCAT_WS('|',COALESCE(PERNR, ''), COALESCE(PERSK, ''),COALESCE(TRY_CAST(ORGEH AS STRING), ''),COALESCE(PLANS, ''),COALESCE(STELL, ''),COALESCE(SNAME, ''),COALESCE(ENAME, ''),COALESCE(OTYPE, '')),256) AS OA_RECORDID
            FROM temp_pa0001) 
        SELECT  PERSONNEL_NUMBER,EE_SUBGROUP,ORG_UNIT,POSITION,JOB,NAME_SOFTWARE,NAME_FORMATTED,OBJ_TYPE,START_DATE,END_DATE,COALESCE(NULL, CURRENT_TIMESTAMP()), OA_RECORDID FROM pa0001 WHERE pa0001_duplicate = 1

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)

In [ ]:
%%sql 
select * from sap_pa0001 limit 5;
select * from sap_hrp1001 limit 5;
select * from sap_hrp1000 limit 5;

StatementMeta(, 1d67b427-9a38-4ee1-8fc6-dfcde996ad9b, -1, Cancelled, , Cancelled)